# Why Jupyter Notebook ?

As most of the evaluation of model preformance will require human juding, It's better to experiment in Jupyter Notebook for now.

Of course metrics will be put into account, but I noticed with the inital experiment that the model generates white pixels "Background was white" everywhere so it decreases the loss. basically the model learnt to generate the background not the required task. it found it easier that way to decrease the loss :D

In [19]:
import tensorflow as tf
from Utils.model_builder.tf_models import model_01
import mlflow
from Utils.mlflow.mlflow import mlflow_track

2022-09-29 17:19:26.068499: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-29 17:19:26.903891: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-29 17:19:28.637127: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvrtc.so.11.1: cannot open shared object file: No such file or directory
2022-09-29 17:19:28.639732: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvrtc.so.11.1: cannot open shared object file: No such file or direc

In [21]:
from Utils.config import load_config_file
config_file = load_config_file()

In [22]:
track_uri = config_file["mlflow"]["track_uri"]
data_version = config_file["data_versions"][-1] # -1 to get the latest only without having to hardcode it in here,
                                                # "Assuming that the last will always will be the updated one"
epochs = config_file["train_config"]["epochs"]


In [24]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
callbacks = []
callbacks.append(EarlyStopping(monitor='val_loss',patience=config_file["train_config"]["earl_stop"],
                verbose=1,restore_best_weights=True))
callbacks.append(ReduceLROnPlateau(monitor='val_loss',
                patience=config_file["train_config"]["lr_reduce_pati"],verbose=1))

In [ ]:
mlflow.set_tracking_uri(track_uri)
mlflow.set_experiment("X_X_images")

In [ ]:
@mlflow_track
def exp_x_x_images(dataset,data_version=data_version):
    """I would like to try generating the same images so that the model can learn the feature of our dataset,
    then leverage this learning with our task"""
    mlflow.tensorflow.autolog()
    mlflow.log_param("Data Version", data_version)
    lr = 0.0001
    loss = config_file["train_config"]["loss"]
    metric = config_file["train_config"]["metric"]
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

    model = model_01.model(loss_func=loss, optimizer=optimizer, metrics=metric).get_model()
    mlflow.log_param("Model Module", "model_01")
    history = model.fit(dataset,epochs=epochs,callbacks=callbacks)
    mse = model.evalute()
    mlflow.log_metric("mse", mse)
    
    return history

In [ ]:
#TODO seek hyperparameter tuning

In [ ]:
mlflow.set_experiment("X_Y_images")